In [60]:
import sh
import json
import os
import glob2
from collections import Counter
import csv
import sys
import re
from collections import namedtuple
from types import SimpleNamespace

In [2]:
def print_progress(i, l, step=1):
    if i % step == 0:
            print("\r>> {}/{}".format(i + 1, l), end="")
            sys.stdout.flush()

In [3]:
if not os.path.exists(os.getcwd() + '/lucene-solr'):
    print("Path doesn\'t exists, cloning repo:")
    sh.git.clone("https://github.com/apache/lucene-solr.git")
else:
    print("Repository exists")

Repository exists


In [4]:
git = sh.git.bake(_cwd='lucene-solr')
git.checkout("trunk")
git.status()

On branch trunk
Your branch is up-to-date with 'origin/trunk'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	diff.txt

nothing added to commit but untracked files present (use "git add" to track)

In [5]:
commits = []

log = git.log("--reverse", "--pretty=format:%H")
for c_hash in log:
    if(c_hash.endswith('\n')):
        c_hash = c_hash[:-1]
    commits.append(c_hash)
len(commits)

16125

In [6]:
#This is specific for LUCENE, it removes the commits that are only SOLR related
lucene_commits = []
l = len(commits)
for idx, c in enumerate(commits):
    
    print_progress(idx, l, 10)
        
    details = git.show("--name-only","--pretty=format:%B", c)
    files = git.show("--name-only","--pretty=format:",c)
    
    #Discarding commits that modify only solr files
    only_solr = True
    for file in files:
        if file.startswith('lucene'):
            only_solr = False

    if only_solr is True:
        continue
    
    lucene_commits.append(c)
    
len(lucene_commits)

>> 16121/16125

8316

In [7]:
# '@@ -[0-9]+,[0-9]+ \+[0-9]+,[0-9]+ @@'  REGEX TO MATCH THE DIFF LINE INDEXES
#def count_changes(commit_hash, file_matching_regexp):
#    diff = str(git.diff("--stat", commit_hash + "^", commit_hash))
#    
#    changes_count = dict() #This will be filled with tuples: (lines_added, lines_deleted)
#    for l in diff.split('\n'):
#        split = l.split("|")
#        
#        m = re.findall(file_matching_regexp, split[0]) # Only lucene files
#        if len(m) == 0:
#            continue
#        file_name = m[0]
#        
#        m = re.findall("-+", split[1])
#        if len(m) == 0:
#            deleted = 0
#       else:
#           deleted = len(m[0])
#       
#       m = re.findall("\++", split[1])
#       if len(m) == 0:
#           added = 0
#       else:
#           added = len(m[0])

#        changes_count[file_name] = ((added, deleted))
    
#    return changes_count

In [51]:
Changes = namedtuple('Changes', 'added, deleted')

def count_changes(commit_hash, file_matching_regexp):
    files = git.show("--name-only","--pretty=format:",commit_hash)
    changes_count = dict() #This will be filled with tuples: (lines_added, lines_deleted)
    for f in files:
        f = f[:-1]
        m = re.findall(file_matching_regexp, f)
        if len(m) == 0:
                continue
        diff = str(git.diff("--stat", commit_hash + "^", commit_hash, "--", f))
        m = re.findall("[0-9]+ insertions", diff)
        if len(m) == 0:
            added = 0
        else:
            added = int(m[0].split(' ')[0])
        m = re.findall("[0-9]+ deletions", diff)
        if len(m) == 0:
            deleted = 0
        else:
            deleted = int(m[0].split(' ')[0])

        changes_count[f] = Changes(added, deleted)

    return changes_count

In [81]:
#TEST
#commit_hash = lucene_commits[4]
print(commit_hash)
commit_hash = "788a61934e8b199fd8614e6a008f4ea9cb456003"
#command = "git diff --stat " + commit_hash + "^ " + commit_hash
s = str(git.show('--name-only','--format="%aN <%aE>"', commit_hash)).split('\n')[0].split('<')
author_name = s[0][1:-1]
author_email = s[1][:-2]
file_changes = count_changes(commit_hash, 'lucene.*\.java')
date = git.show("-s", "--format=%ci", commit_hash)
print(author_email)
print(author_name)
print(date)
file_changes

788a61934e8b199fd8614e6a008f4ea9cb456003
rmuir@apache.org
Robert Muir
2010-06-01 10:35:13 +0000



{'lucene/contrib/benchmark/src/test/org/apache/lucene/benchmark/byTask/TestPerfTasksLogic.java': Changes(added=6, deleted=19),
 'lucene/contrib/highlighter/src/java/org/apache/lucene/search/highlight/Highlighter.java': Changes(added=3, deleted=3),
 'lucene/contrib/highlighter/src/java/org/apache/lucene/search/highlight/QueryScorer.java': Changes(added=4, deleted=4),
 'lucene/contrib/highlighter/src/java/org/apache/lucene/search/highlight/QueryTermScorer.java': Changes(added=4, deleted=4),
 'lucene/contrib/highlighter/src/java/org/apache/lucene/search/highlight/SimpleSpanFragmenter.java': Changes(added=4, deleted=4),
 'lucene/contrib/highlighter/src/java/org/apache/lucene/search/highlight/TokenGroup.java': Changes(added=4, deleted=4),
 'lucene/contrib/highlighter/src/java/org/apache/lucene/search/highlight/TokenSources.java': Changes(added=5, deleted=6),
 'lucene/contrib/highlighter/src/java/org/apache/lucene/search/highlight/TokenStreamFromTermPositionVector.java': Changes(added=5, del

In [58]:
def update_counters(file_dict, file_name, author, changes):
#Create/update file entries in the dict()
    if file_dict.get(file_name) == None:
        file_dict[file_name] = SimpleNamespace()
        f = file_dict[file_name]
        f.author_dict = dict()
        f.tot_lines_added = 0
        f.tot_lines_deleted = 0
        f.tot_commits = 0
         
    f = file_dict[file_name]
    f.tot_lines_added += changes.added
    f.tot_lines_deleted += changes.deleted
    f.tot_commits += 1
     
#Create/update author entries in the dict() for that file
    if f.author_dict.get(author) == None:
        f.author_dict[author] = SimpleNamespace()
        a = f.author_dict[author]
        a.lines_added = 0
        a.lines_deleted = 0
        a.commits = 0
        
    a = f.author_dict[author]
    a.lines_added += changes.added
    a.lines_deleted += changes.deleted
    a.commits += 1

In [93]:
def extract_bug(file):
    if not file.endswith('.json'):
            return None
    else:
        bug_json_string = open("issue_LUCENE/" + file).read()
        bug = json.loads(bug_json_string)
        bug_fields = bug.get('fields')
        
        if bug_fields['issuetype']['name'] != 'Bug':
            return None
   
        if bug_fields['resolution'] == None:
            return None 
                
        if bug_fields['resolution']['name'] != 'Fixed':
            return None
    return bug

In [94]:
#Extracting all the fixed bugs from the bug repo
bugs = dict() #Dictionary of bugs indexed by LUCENE+ID
dir_list = os.listdir(os.getcwd() + "/issue_LUCENE")
l = len(dir_list)
print("Progress:")
for idx, file in enumerate(dir_list):
    print_progress(idx,l,10)
    bug = extract_bug(file)
    if(bug == None):
        continue
    bugs[bug['key'].upper()] = bug

Progress:
>> 6641/6641

In [98]:
def get_fixed_bugs(commit_hash):
    details = git.show("--name-only","--pretty=format:%B", commit_hash)
    init = details.split(':')[0].upper()
    f_keys = [] 
    if init.startswith('LUCENE') or init.startswith('SOLR'):  #Because it could be SOLR-123, LUCENE-234, ...
        if init.find(', ') != -1:
            keys = init.split(', ')
        elif init.find(',') != -1:
            keys = init.split(',')
        else:
            keys = [init.split(' ')[0]]
               
        for k in keys:
            if bugs.get(k) != None:
                #Then it fixes a bug
                f_keys.append(k)
    return f_keys

In [117]:
def get_affected_versions(fixed_bug_keys):
    affected_versions = []
    for b in fixed_bug_keys:
        versions = bugs[b]['fields']['versions']
        if len(versions) == 0:
            continue
        else:    
            for v in versions:
                n = v['name']
                affected_versions.append(n)
    return affected_versions
#get_affected_versions(['LUCENE-2244', 'LUCENE-3255'])

In [164]:
l = len(lucene_commits)

file_dict = dict()

#FileCounter = namedtuple('FileCounter', '''author_dict, 
#                                             tot_lines_added, tot_lines_deleted, 
#                                             tot_commits''')
#AuthorCounter = namedtuple('AuthorCounter', '''lines_added, 
#                                                 lines_deleted, 
#                                                 commits''')
columns = '''commit_hash, commit_date, file, author, 
commit_file_lines_added, commit_file_lines_deleted, 
author_file_tot_lines_added, author_file_tot_lines_deleted, author_file_commits, 
file_tot_lines_added, file_tot_lines_deleted, file_tot_commits, 
bug_fix, fixed_bugs, affected_versions'''

#Row = namedtuple('Row', columns)
filename = 'out.csv'
file = open(filename, 'w')
wr = csv.writer(file, quoting=csv.QUOTE_ALL)
wr.writerow(columns.replace('\n', '').split(', '))

for idx, commit_hash in enumerate(lucene_commits):
    print_progress(idx, l)
    s = str(git.show("--name-only", '--format="%aN <%aE>"', commit_hash)).split('\n')[0].split('<')
    author_name = s[0][1:-1]
    #author_email = s[1][:-2]
    date = str(git.show("-s", "--format=%ci", commit_hash))[:-1].replace("\n", '')
    file_changes = count_changes(commit_hash, 'lucene.*\.java')
    
    #Checking if it is a bug_fix and the affected versions
    bug_fix = 0
    f_keys = get_fixed_bugs(commit_hash)
    if len(f_keys) != 0:
        bug_fix = 1
            
    for file_name, changes in file_changes.items():
        #print(key, value)
        update_counters(file_dict, file_name, author_name, changes)
        f = file_dict[file_name]
        for author, author_counter in f.author_dict.items():
            wr.writerow([commit_hash, #commit_hash
                         date, #commit_date
                         file_name, #file
                         author, #author_name
                         changes.added, #commit_file_lines_added
                         changes.deleted, #commit_file_lines_deleted
                         author_counter.lines_added, #author_file_tot_lines_added
                         author_counter.lines_deleted, #author_file_tot_lines_deleted
                         author_counter.commits, #author_file_commits
                         f.tot_lines_added, #file_tot_lines_added
                         f.tot_lines_deleted, #file_tot_lines_deleted
                         f.tot_commits, #file_tot_commits
                         bug_fix, #bug_fix
                         f_keys, #fixed_bugs
                         get_affected_versions(f_keys) #affected_versions
                        ]) 
        
        #Now I write the csv line:

        #print()
        #print(file_dict.get(key))
file.close()

>> 8316/8316

In [220]:
import pandas as pd
df = pd.read_csv(filename)
df

,commit_hash,commit_date,file,author,commit_file_lines_added,commit_file_lines_deleted,author_file_tot_lines_added,author_file_tot_lines_deleted,author_file_commits,file_tot_lines_added,file_tot_lines_deleted,file_tot_commits,bug_fix,fixed_bugs,affected_versions
0,3ee0ace1ba6b9bff3ffaa278c0bba07e6064057d,2010-03-17 20:51:29 +0000,lucene/backwards/src/java/org/apache/lucene/in...,Mark Robert Miller,89,0,89,0,1,89,0,1,0,[],[]
1,3ee0ace1ba6b9bff3ffaa278c0bba07e6064057d,2010-03-17 20:51:29 +0000,lucene/backwards/src/java/org/apache/lucene/in...,Mark Robert Miller,25,0,25,0,1,25,0,1,0,[],[]
2,3ee0ace1ba6b9bff3ffaa278c0bba07e6064057d,2010-03-17 20:51:29 +0000,lucene/backwards/src/java/org/apache/lucene/se...,Mark Robert Miller,423,0,423,0,1,423,0,1,0,[],[]
3,3ee0ace1ba6b9bff3ffaa278c0bba07e6064057d,2010-03-17 20:51:29 +0000,lucene/contrib/surround/src/java/org/apache/lu...,Mark Robert Miller,109,0,109,0,1,109,0,1,0,[],[]
4,3ee0ace1ba6b9bff3ffaa278c0bba07e6064057d,2010-03-17 20:51:29 +0000,lucene/contrib/analyzers/common/src/java/org/a...,Mark Robert Miller,50,0,50,0,1,50,0,1,0,[],[]
5,3ee0ace1ba6b9bff3ffaa278c0bba07e6064057d,2010-03-17 20:51:29 +0000,lucene/contrib/benchmark/src/java/org/apache/l...,Mark Robert Miller,134,0,134,0,1,134,0,1,0,[],[]
6,3ee0ace1ba6b9bff3ffaa278c0bba07e6064057d,2010-03-17 20:51:29 +0000,lucene/src/test/org/apache/lucene/util/TestPri...,Mark Robert Miller,119,0,119,0,1,119,0,1,0,[],[]
7,3ee0ace1ba6b9bff3ffaa278c0bba07e6064057d,2010-03-17 20:51:29 +0000,lucene/contrib/spatial/src/test/org/apache/luc...,Mark Robert Miller,140,0,140,0,1,140,0,1,0,[],[]
8,3ee0ace1ba6b9bff3ffaa278c0bba07e6064057d,2010-03-17 20:51:29 +0000,lucene/backwards/src/java/org/apache/lucene/in...,Mark Robert Miller,422,0,422,0,1,422,0,1,0,[],[]
9,3ee0ace1ba6b9bff3ffaa278c0bba07e6064057d,2010-03-17 20:51:29 +0000,lucene/contrib/benchmark/src/java/org/apache/l...,Mark Robert Miller,145,0,145,0,1,145,0,1,0,[],[]


In [221]:
df2 = pd.read_csv('implicated_files.csv')
df2

,commit_hash,file,implicated
0,ee69d9294ad820ce3c6c6225d1c3229f3d5aa411,lucene/suggest/src/java/org/apache/lucene/sear...,1
1,ee69d9294ad820ce3c6c6225d1c3229f3d5aa411,lucene/core/src/java/org/apache/lucene/util/fs...,1
2,ee69d9294ad820ce3c6c6225d1c3229f3d5aa411,lucene/suggest/src/test/org/apache/lucene/sear...,1
3,5660a0e2d1969921dbe321c33491baabc20db80a,lucene/core/src/test/org/apache/lucene/search/...,1
4,8461a847c981b9cf5f9d6453349dfc5a48b8fa36,lucene/analysis/phonetic/src/test/org/apache/l...,1
5,7e4580920093ab9312d2b9889b74527d07cbf7d4,lucene/core/src/java/org/apache/lucene/codecs/...,1
6,7e4580920093ab9312d2b9889b74527d07cbf7d4,lucene/codecs/src/java/org/apache/lucene/codec...,1
7,7e4580920093ab9312d2b9889b74527d07cbf7d4,lucene/core/src/java/org/apache/lucene/codecs/...,1
8,f0f1569b41b5b164c4202b97e6cbd2c7abfdb0db,lucene/queries/src/test/org/apache/lucene/quer...,1
9,3acf2ce9ac0a85f86e143ae7dbaa920d987624c4,lucene/core/src/test/org/apache/lucene/index/T...,1


In [222]:
df = df.merge(df2, on=['commit_hash', 'file'], how='left').fillna(0)

In [237]:
df.to_csv('lucene_dataset.csv')

In [236]:
df[df['commit_hash'] == '2747cc432c7ea2f61e42a78b3bf8296a88c5879b'] #Example of commit

,commit_hash,commit_date,file,author,commit_file_lines_added,commit_file_lines_deleted,author_file_tot_lines_added,author_file_tot_lines_deleted,author_file_commits,file_tot_lines_added,file_tot_lines_deleted,file_tot_commits,bug_fix,fixed_bugs,affected_versions,implicated
252597,2747cc432c7ea2f61e42a78b3bf8296a88c5879b,2015-11-26 04:56:47 +0000,lucene/grouping/src/java/org/apache/lucene/sea...,Adrien Grand,4,3,3,4,2,105,22,13,0,[],[],0
252598,2747cc432c7ea2f61e42a78b3bf8296a88c5879b,2015-11-26 04:56:47 +0000,lucene/grouping/src/java/org/apache/lucene/sea...,Simon Willnauer,4,3,0,0,1,105,22,13,0,[],[],0
252599,2747cc432c7ea2f61e42a78b3bf8296a88c5879b,2015-11-26 04:56:47 +0000,lucene/grouping/src/java/org/apache/lucene/sea...,Uwe Schindler,4,3,0,0,1,105,22,13,0,[],[],0
252600,2747cc432c7ea2f61e42a78b3bf8296a88c5879b,2015-11-26 04:56:47 +0000,lucene/grouping/src/java/org/apache/lucene/sea...,Ryan Ernst,4,3,2,2,1,105,22,13,0,[],[],0
252601,2747cc432c7ea2f61e42a78b3bf8296a88c5879b,2015-11-26 04:56:47 +0000,lucene/grouping/src/java/org/apache/lucene/sea...,Michael McCandless,4,3,10,11,3,105,22,13,0,[],[],0
252602,2747cc432c7ea2f61e42a78b3bf8296a88c5879b,2015-11-26 04:56:47 +0000,lucene/grouping/src/java/org/apache/lucene/sea...,Robert Muir,4,3,86,2,3,105,22,13,0,[],[],0
252603,2747cc432c7ea2f61e42a78b3bf8296a88c5879b,2015-11-26 04:56:47 +0000,lucene/grouping/src/java/org/apache/lucene/sea...,Chris M. Hostetter,4,3,0,0,1,105,22,13,0,[],[],0
252604,2747cc432c7ea2f61e42a78b3bf8296a88c5879b,2015-11-26 04:56:47 +0000,lucene/grouping/src/java/org/apache/lucene/sea...,David Wayne Smiley,4,3,4,3,1,105,22,13,0,[],[],0
252605,2747cc432c7ea2f61e42a78b3bf8296a88c5879b,2015-11-26 04:56:47 +0000,lucene/grouping/src/test/org/apache/lucene/sea...,Adrien Grand,0,0,10,14,5,1981,630,46,0,[],[],0
252606,2747cc432c7ea2f61e42a78b3bf8296a88c5879b,2015-11-26 04:56:47 +0000,lucene/grouping/src/test/org/apache/lucene/sea...,Ryan Ernst,0,0,10,10,4,1981,630,46,0,[],[],0
